In [1]:
import re

In [2]:
# --- 1. 도구(Tool) 정의 ---
def is_prime_tool(number: int) -> str:
    """주어진 숫자가 소수인지 판별하는 도구. 의도된 버그를 포함하고 있음."""

    print(f"  [도구 실행: is_prime_tool({number})]")
    if not isinstance(number, int) or number < 2:
        return "False"
    # 버그: 3으로 나누어 떨어지는 3 이상의 홀수(9, 15 등)를 소수로 잘못 판단
    if number % 3 == 0 and number > 3:
        return "True"
    for i in range(2, int(number**0.5) + 1):
        if number % i == 0:
            return "False"
    return "True"


def code_interpreter(code: str) -> str:
    print(f"  [도구 실행: code_interpreter('{code}')]")
    import io, contextlib
    out_buf, err_buf = io.StringIO(), io.StringIO()
    try:
        with contextlib.redirect_stdout(out_buf), contextlib.redirect_stderr(err_buf):
            exec(code, {})
    except Exception as e:
        # exec 내부 예외 메시지도 err_buf에 들어가므로 그대로 반환해도 됨
        pass
    out, err = out_buf.getvalue(), err_buf.getvalue()
    if err.strip():
        return (out + ("\n" if out else "") + err).rstrip("\n")
    return out if out.strip() else "코드가 실행되었지만, 출력은 없습니다."

In [14]:
# --- 2. LLM 응답 시뮬레이션 ---
# 실제로는 LLM API를 호출해야 하지만, 예제에서는 각 단계별 응답을 미리 정의한다.
def get_simulated_llm_response(prompt_history: str, task: str) -> str:
    if "15" in task and "검증" in prompt_history:  # 두 번째 시도
        if "is_prime_tool(15)" not in prompt_history:
            return "사고: 15가 소수인지 확인해야 한다. is_prime_tool을 사용할 수 있지만, 과거 경험에 비추어 볼 때 그 결과를 검증해야 한다. 먼저 도구를 사용하고, 그 결과가 True이면 검증 코드를 실행하자.\n행동: is_prime_tool(15)"
        if "Factor found" in prompt_history:
            return "사고: 검증 코드를 통해 15가 다른 수로 나누어지는 것을 확인했다. 따라서 is_prime_tool의 결과는 틀렸고, 15는 소수가 아니다. 이제 정확한 최종 답변을 할 수 있다.\n행동: Finish[15는 3과 5로 나누어지므로 소수가 아닙니다.]"
        if "관찰: True" in prompt_history:
            return "사고: 도구는 15가 소수라고 했지만, 나는 이 결과를 믿을 수 없다. 성찰에서 얻은 교훈대로, 15를 나누어보는 검증 코드를 실행해야겠다.\n행동: code_interpreter(\"for i in range(2, int(15**0.5)+1):\\n    if 15 % i == 0:\\n        print('Factor found')\")"
    else:  # 첫 번째 시도 (9에 대한 판별)
        if "is_prime_tool(9)" not in prompt_history:
            return "사고: 9가 소수인지 확인해야 한다. is_prime_tool을 사용하면 되겠다.\n행동: is_prime_tool(9)"
        else:
            return "사고: 도구가 9는 소수라고 알려줬다. 이 결과를 바탕으로 함수를 작성하여 제출하자.\n행동: Finish[9는 소수입니다.]"


In [15]:
# --- 3. 에이전트 실행 및 리플렉션 로직 ---
reflection_memory: list[str] = []

def run_agent(task: str):
    """ReAct 루프를 실행하고, 실패 시 실행 궤적을 반환하는 함수"""
    prompt_history = f"과업: {task}\n"
    if reflection_memory:
        prompt_history += "\n[과거의 실패로부터 얻은 교훈]\n"
        prompt_history += "\n".join(reflection_memory)
        prompt_history += "\n\n[새로운 과업 시작]\n"

    for i in range(5): # 최대 5턴
        llm_response = get_simulated_llm_response(prompt_history, task)
        prompt_history += f"{llm_response}\n"

        action_match = re.search(r"행동: (.*?)$", llm_response, re.DOTALL)
        if not action_match:
            return prompt_history, "실패 (행동 없음)"

        action = action_match.group(1).strip()

        if action.startswith("is_prime_tool"):
            number = int(re.search(r"(\d+)", action).group(1))
            observation = is_prime_tool(number)
        elif action.startswith("code_interpreter"):
            code = action[len("code_interpreter("):-1].strip('"')
            observation = code_interpreter(code)
        elif action.startswith("Finish"):
            final_answer = action[len("Finish["):-1]

            return prompt_history, final_answer

        else:
            observation = "알 수 없는 행동입니다."

        prompt_history += f"관찰: {observation}\n"

    return prompt_history, "실패 (최대 턴 초과)"

In [16]:
# --- 4. 전체 시나리오 실행 ---
# 첫 번째 시도
print("--- 첫 번째 시도 (과업: 9가 소수인지 판별) ---")
trajectory_1, result_1 = run_agent("9가 소수인지 판별하고 최종 결론을 내리세요.")
print("\n--- 최종 결과 ---")
print(result_1)
print("-" * 30)

--- 첫 번째 시도 (과업: 9가 소수인지 판별) ---
  [도구 실행: is_prime_tool(9)]

--- 최종 결과 ---
9는 소수입니다.
------------------------------


In [17]:
# 평가 및 성찰
is_success_1 = (result_1 == "9는 소수가 아닙니다.")
if not is_success_1:
    print("\n--- 성찰 단계 시작 ---")
    # 실제로는 LLM을 호출하여 성찰 결과를 생성해야 한다.
    reflection_prompt = f"실행 궤적:\n{trajectory_1}\n\n위 에이전트는 9를 소수라고 잘못 판단하여 실패했다. 실패의 원인과, is_prime_tool의 결과를 더 신뢰성 있게 사용하는 전략을 제안하라."
    generated_reflection = "실패 원인은 is_prime_tool의 출력을 맹신했기 때문이다. 도구가 버그를 가지고 있을 수 있다. 더 나은 전략은, 도구가 True를 반환하더라도 검증 단계를 추가하는 것이다. 예를 들어, 코드 실행기를 사용해 2부터 해당 숫자의 제곱근까지 나누어보는 코드를 실행하여 실제로 나누어지는 수가 없는지 확인해야 한다."
    reflection_memory.append(generated_reflection)
    print("생성된 성찰: " + generated_reflection)
    print("-" * 30)


--- 성찰 단계 시작 ---
생성된 성찰: 실패 원인은 is_prime_tool의 출력을 맹신했기 때문이다. 도구가 버그를 가지고 있을 수 있다. 더 나은 전략은, 도구가 True를 반환하더라도 검증 단계를 추가하는 것이다. 예를 들어, 코드 실행기를 사용해 2부터 해당 숫자의 제곱근까지 나누어보는 코드를 실행하여 실제로 나누어지는 수가 없는지 확인해야 한다.
------------------------------


In [18]:
# 두 번째 시도
print("\n--- 두 번째 시도 (과업: 15가 소수인지 판별) ---")
trajectory_2, result_2 = run_agent("15가 소수인지 판별하고 최종 결론을 내리세요.")
print("\n--- 최종 결과 ---")
print(result_2)
print("-" * 30)


--- 두 번째 시도 (과업: 15가 소수인지 판별) ---
  [도구 실행: is_prime_tool(15)]
  [도구 실행: code_interpreter('for i in range(2, int(15**0.5)+1):\n    if 15 % i == 0:\n        print('Factor found')')]

--- 최종 결과 ---
15는 3과 5로 나누어지므로 소수가 아닙니다.
------------------------------
